In [ ]:
!pip install torch transformers datasets evaluate accelerate timm ipykernel ipywidgets IProgress scikit-learn scipy peft bitsandbytes trl streamlit

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
# this restricts libs to only only allow specified gpus to be used
import torch
import os
import numpy as np
import random
CUDA_DEVICE_IDX = 5
os.environ["CUDA_VISIBLE_DEVICES"] = f"{CUDA_DEVICE_IDX}"
device_map = f"cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_map)

def set_seed(random_seed: int = 42):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

# Choosing model

In [3]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
checkpoint = "ai-forever/rugpt3large_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

# Dataset preprocessing

In [4]:
# uncomment this to download Dostaevskiy's text corpus
# !wget https://gitlab.com/z00logist/artificial-dostoevsky/-/raw/main/data/corpus.txt

In [5]:
raw_text = open('corpus.txt').read()
print('Text example:', raw_text[:300])
print('Length of full corpus:', len(raw_text))

Text example: Ох уж эти мне сказочники! Нет чтобы написать что-нибудь полезное, приятное, усладительное, а то всю подноготную в земле вырывают!.. Вот уж запретил бы им писать! Ну, на что это похоже: читаешь... невольно задумаешься, а там всякая дребедень и пойдет в голову; право бы, запретил им писать; так-таки п
Length of full corpus: 9652088


# Let's make Train/Validation/Test splits

In [6]:
from collections import OrderedDict
from datasets import Dataset, DatasetDict
# let's divide text into chunks of roughly 10.000 chars,
# then shuffle them between splits
document_size = 10000
print('Number of documents:', len(raw_text) / document_size)

def get_split_idx(sizes: list | tuple, size: int):
    """Returns idx for split according to distributions of sizes"""
    idx = torch.arange(0, size)
    probabilities = torch.tensor(sizes)
    probabilities /= probabilities.sum()
    split_mask = torch.multinomial(probabilities, num_samples=size, replacement=True)
    return tuple((idx[split_mask == i].numpy().tolist() for i in range(len(sizes))))

def get_dataset_dict(text: str, sizes_dict: OrderedDict):
    split_idx = get_split_idx(list(sizes_dict.values()), int(len(text) / document_size))
    split_idx = {split: split_idx[i] for i, split in enumerate(sizes_dict)}
    datasets = DatasetDict() 
    for split, idx in split_idx.items():
        datasets[split] = Dataset.from_dict({'text': [text[i * document_size: i * document_size + document_size] for i in idx], 'start_idx': [i * document_size for i in idx]})
    return datasets

set_seed(52)
datasets = get_dataset_dict(raw_text, OrderedDict([('train', 0.8), ('validation', 0.1), ('test', 0.1)]))
datasets

Number of documents: 965.2088


DatasetDict({
    train: Dataset({
        features: ['text', 'start_idx'],
        num_rows: 757
    })
    validation: Dataset({
        features: ['text', 'start_idx'],
        num_rows: 91
    })
    test: Dataset({
        features: ['text', 'start_idx'],
        num_rows: 117
    })
})

In [7]:
# let's check (chunk_sizes are not all the same though)
print(datasets['train'][7]['text'])
print(f"start_idx: {datasets['train'][7:10]['start_idx']}, chunk_sizes: {list(map(len, datasets['train'][7:10]['text']))}")

а! Цена всех одиннадцати книг, присовокупив сюда издержки на переплет, была по крайней мере рублей шестьдесят. Где взять денег? Я думала-думала и не знала, на что решиться. У матушки просить не хотелось. Конечно, матушка мне непременно бы помогла; но тогда все бы в доме узнали о нашем подарке; да к тому же этот подарок обратился бы в благодарность, в плату за целый год трудов Покровского. Мне хотелось подарить одной, тихонько от всех. А за труды его со мною я хотела быть ему навсегда одолженною без какой бы то ни было уплаты, кроме дружбы моей. Наконец я выдумала, как выйти из затруднения. Я знала, что у букинистов в Гостином дворе можно купить книгу иногда в полцены дешевле, если только поторговаться, часто малоподержанную и почти совершенно новую. Я положила непременно отправиться в Гостиный двор. Так и случилось; назавтра же встретилась какая-то надобность и у нас и у Анны Федоровны. Матушке понездоровилось, Анна Федоровна очень кстати поленилась, так что пришлось все поручения возл

In [8]:
# tokenizing before chunking
def tokenize_function(examples):
    return tokenizer(examples['text'])
tokenized_datasets = datasets.map(tokenize_function, remove_columns=['start_idx', 'text'], batched=True).remove_columns('attention_mask')
tokenized_datasets

Map:   0%|          | 0/757 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2617 > 2048). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 757
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 91
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 117
    })
})

In [10]:
# let's see what we get
print(f"document_sizes: {list(map(len, tokenized_datasets['train'][0:10]['input_ids']))}")

document_sizes: [2617, 2566, 2578, 2529, 2380, 2325, 2352, 2376, 2598, 2631]


In [ ]:
# now let's take these documents and chunk them up each, chunks will be of size 1024. In order to fit in RAM restrictions, we'll use IterableDataset and use generator
from datasets import IterableDataset, IterableDatasetDict
from random import randint
from functools import partial
chunk_size = 1024

def chunk_generator(dataset: Dataset, sequential: bool = False):
    """Generator that (maybe randomly) picks documents from the dataset and then random chunk of size chunk_size"""
    if sequential:
        for document_idx in range(len(dataset)):
            tokens = dataset[document_idx]['input_ids']
            i = randint(0, len(tokens) - chunk_size)
            yield {'input_ids': tokens[i: i + chunk_size]}
    else:
        while True:
            document_idx = randint(0, len(dataset) - 1)
            tokens = dataset[document_idx]['input_ids']
            i = randint(0, len(tokens) - chunk_size)
            yield {'input_ids': tokens[i: i + chunk_size]}

set_seed(2718)
# it's iterable, so it's light-weight
chunked_data = IterableDatasetDict()
for split in tokenized_datasets:
    dataset = tokenized_datasets[split]
    chunked_data[split] = (IterableDataset if split == 'train' else Dataset).from_generator(
        partial(chunk_generator, dataset=dataset, sequential=split != 'train')
    )
tokenized_datasets = chunked_data
chunked_data

Generating train split: 0 examples [00:00, ? examples/s]

IterableDatasetDict({
    train: IterableDataset({
        features: Unknown,
        num_shards: 1
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 91
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 117
    })
})

In [12]:
import psutil
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")


RAM used: 1260.62 MB


In [13]:
# let's check dtype of the dataset
type(list(tokenized_datasets['train'].take(1))[0]['input_ids'])

list

In [14]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # does job for creating attention_mask and labels for us (it just copies inputs for labels, bcause shifting happens inside the model)

In [15]:
out = data_collator(list(tokenized_datasets["train"].take(5)))
out

{'input_ids': tensor([[   16,  1012,   784,  ...,   669,  4223,   309],
        [13134,  2625,   519,  ...,  1441,    16,   687],
        [  299,  1186,  1248,  ...,   433,   968,    18],
        [   16,   670,    16,  ...,  1335, 35338,    18],
        [ 2754,  5804,   289,  ...,   503,  1042,    18]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[   16,  1012,   784,  ...,   669,  4223,   309],
        [13134,  2625,   519,  ...,  1441,    16,   687],
        [  299,  1186,  1248,  ...,   433,   968,    18],
        [   16,   670,    16,  ...,  1335, 35338,    18],
        [ 2754,  5804,   289,  ...,   503,  1042,    18]])}

# Training

In [16]:
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

## LoRA

In [28]:
from transformers import BitsAndBytesConfig
from trl import SFTTrainer

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    device_map=device_map
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16, # 8-256
    lora_alpha=32, # (>= r) alpha/r ~ 2-4
    target_modules=["c_attn", "c_proj"], 
    lora_dropout=0.05, # 0.01-0.2
    # bias="none", # train only LoRA weights, no biases 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

In [29]:
model.print_trainable_parameters()

trainable params: 6,488,064 || all params: 766,788,096 || trainable%: 0.8461


In [ ]:
training_args = TrainingArguments(
    output_dir="./rugpt3large_lora_results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    fp16=True,
    max_grad_norm=0.5,
    gradient_checkpointing=True,
    lr_scheduler_type="cosine",
    max_steps=50,
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=5,
    logging_dir="./rugpt3large_lora_logs",
    eval_strategy="steps",
    eval_steps=5,
    save_strategy="steps",
    save_steps=1000,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    peft_config=peft_config,
    processing_class=tokenizer
    # packing=True
)

Truncating eval dataset:   0%|          | 0/91 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [31]:
set_seed(11)
trainer.train()

/home/illjahunovan/poetry_env/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
5,3.314500,3.280031
10,3.320500,3.257753
15,3.278800,3.243949
20,3.275000,3.235051
25,3.295000,3.229133
30,3.256300,3.225114
35,3.283900,3.222836
40,3.250100,3.221640
45,3.264000,3.221438
50,3.244200,3.220997


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=50, training_loss=3.277965326309204, metrics={'train_runtime': 825.3778, 'train_samples_per_second': 1.939, 'train_steps_per_second': 0.061, 'total_flos': 6748054644326400.0, 'train_loss': 3.277965326309204})

# Let's test it out (Inference)

In [ ]:
from peft import PeftModel
checkpoint = 'checkpoint-50'
base_model = AutoModelForCausalLM.from_pretrained(checkpoint)
model = PeftModel.from_pretrained(base_model, checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def generate_text(prompt: str, max_length: int = 40, temperature: float = 0.7, top_p: float = 0.9, seed: int | float | None = 52, **kwargs):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    set_seed(seed)
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
            **kwargs
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_text_beam(prompt: str, 
                       max_length: int = 40, 
                       num_beams: int = 10, 
                       num_return_sequences: int = 3, 
                       no_repeat_ngram_size: int = 2, 
                       num_beam_groups: int = 2,
                       diversity_penalty: float = 1.0,
                       seed: int | float | None = 52,
                       **kwargs):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    set_seed(seed)
    model.eval()
    with torch.no_grad():
        sequences = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_beams=num_beams, # beam width 
            early_stopping=True, # stop when all beams finish
            no_repeat_ngram_size=no_repeat_ngram_size, # n-gram penalties, technique to avoid repetition
            num_return_sequences=num_return_sequences, # return top k sequences
            num_beam_groups=num_beam_groups, # divides num_beams to this number groups. two groups differ from each other. greatly combines with diversity_penalty 
            diversity_penalty=diversity_penalty, # encourages diversity beams (if using num_beam_groups)
            trust_remote_code=True, # beam search backward compatibility
            **kwargs
        )
    return tokenizer.batch_decode(sequences, skip_special_tokens=True)

In [68]:
outputs = generate_text_beam("Быть или не быть?", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=1, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.0,
                       seed=52)

In [69]:
print(outputs[0])

Быть или не быть? Вот в чем вопрос.

— Да, да, конечно, — сказал я, и мы оба засмеялись. Потом я спросил: — А как вы думаете, что будет с нами, если мы все-таки останемся в живых? Что будет, когда мы умрем? Как мы будем жить после смерти? Ведь мы же не знаем, как жить. Мы даже не можем сказать, кто мы такие, откуда мы, куда мы идем, зачем мы


In [ ]:
outputs = generate_text_beam("Бог ему судья!", 
                       max_length=100, 
                       num_beams=15, 
                       num_return_sequences=15, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=3,
                       diversity_penalty=1.0,
                       seed=52)
print(outputs[0], outputs[5], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Бог ему судья!

— Да, да, я знаю, — сказал я, и мы оба засмеялись. Потом, помолчав немного, прибавил: — Я, право, не понимаю, как это могло случиться, что вы так легко поверили этому человеку. Ведь он, по-видимому, сумасшедший. Я не могу себе представить, чтобы вы могли поверить ему, если бы он не был сумасшедшим. Но ведь это не так, правда? Ведь вы не могли бы ему поверить,
______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [ ]:
outputs = generate_text_beam("И все-таки, тварь ли я дрожащая или право имею?", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

И все-таки, тварь ли я дрожащая или право имею?

— Право имеете, ваше сиятельство, право имеете! — с жаром подтвердил он. Да ведь я и сам не знаю, что со мной делается! Я, кажется, с ума сойду, если с вами не поговорю. Вы, может быть, сердитесь на меня, а я не сержусь на вас, потому что я вас уважаю, и вы меня уважаете. Я вас
________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [ ]:
outputs = generate_text_beam("Какое лучшее определение человека?", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Какое лучшее определение человека?
Человек - это то, что он о себе думает.
Что делать, если я не хочу идти в школу, а родители не хотят идти со мной? Что мне делать? (((
Скажи родителям, чтобы они тебя отвели в другую школу. А если они не согласятся, то скажи что-нибудь обидное, например: "Не хочу я в эту школу! " и т. д. Если родители будут настаивать на своём, скажи им,
_________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [85]:
outputs = generate_text_beam("Сомневаюсь на правильном ли я пути...", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

/home/illjahunovan/poetry_env/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Сомневаюсь на правильном ли я пути...
Что делать, если я не знаю, куда идти?
Идти туда, где не знают, что делать.
А если не знаешь, как идти, то и не надо никуда идти. Зачем тогда вообще что-то делать? Зачем вообще жить? Я не хочу жить, я хочу умереть. Я хочу, чтобы меня оставили в покое, и чтобы никто меня не трогал. Мне не нужно ничего, кроме тишины и покоя.
_____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [99]:
outputs = generate_text_beam("Русский весьма часто смеется там, где", 
                       max_length=28, 
                       num_beams=15, 
                       num_return_sequences=15, 
                       no_repeat_ngram_size=3, 
                       num_beam_groups=3,
                       diversity_penalty=1.0,
                       seed=52)
print(outputs[-1], sep='\n' + '_' * 1000 + '\n')

Русский весьма часто смеется там, где говорит по-французски, а по-итальянски не смеётся никогда, потому что не понимает


In [100]:
outputs = generate_text_beam("Жизнь задыхается без", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Жизнь задыхается без воздуха.

— Да, да, я знаю, — сказал он, и глаза его заблестели, как у ребенка, когда он что-нибудь понимает. Он был в восторге от того, что его догадка подтвердилась. Но он не знал, радоваться ли ему или огорчаться, потому что, несмотря на все его старания, он все-таки не мог понять, в чем тут дело. И вдруг он вспомнил, где он и что с ним
_____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [101]:
outputs = generate_text_beam("Главное, самому себе не лгите... ибо лгущий самому себе теряет способность", 
                       max_length=100, 
                       num_beams=10, 
                       num_return_sequences=10, 
                       no_repeat_ngram_size=2, 
                       num_beam_groups=2,
                       diversity_penalty=1.5,
                       seed=52)
print(outputs[0], outputs[-1], sep='\n' + '_' * 1000 + '\n')

Главное, самому себе не лгите... ибо лгущий самому себе теряет способность различать добро и зло, правду и ложь, истину и кривду, добро от зла, и наоборот. Ложь, как и правда, не нуждается в доказательствах, она сама по себе есть доказательство. Истина же, напротив, требует доказательств, ибо без них нет и не может быть истины. Если же вы хотите, чтобы кто-нибудь доказал вам истину, то докажите сначала, что он лгун
_____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

## Some conclusions after experimenting
- LoRA algorithm is amazing for fine-tuning large and medium size llms \
it provides a way to train even when other common memory saving techniques couldn't help out
- adafactor optimizer (though it wasn't used eventually) is a fantastic replacement for AdamW
- gradient_accumulation idea is a real saver for imitating large batch size
- quantinization and fp16 especially are musthave for nlp tasks
- iterable, streaming datasets combining with generators are light-weight and really handy   
- DataCollatorForLanguageModeling, return_overflowing_tokens, return_overflowing_tokens and other stuff from hf libraries rid of so much boilerplate code